In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb

In [2]:
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')

In [3]:
x = train[['income', 'age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'profession', 'city', 'state', 'current_job_years',
       'current_house_years', 'prob']]
y = train['risk_flag']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
X_test = test[['income', 'age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'profession', 'city', 'state', 'current_job_years',
       'current_house_years', 'prob']]

In [4]:
%%time
knn = KNeighborsClassifier(n_neighbors=12)

knn.fit(x_train, y_train)
knn.score(x_test, y_test)

CPU times: user 2min 21s, sys: 236 ms, total: 2min 21s
Wall time: 2min 21s


0.8917619047619048

In [5]:
%%time

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

CPU times: user 21.4 s, sys: 43.7 ms, total: 21.4 s
Wall time: 21.4 s


0.8984285714285715

In [6]:
%%time 
from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg = LogisticRegression()
#fit the model to the training data
log_reg.fit(x_train, y_train)
log_reg.score(x_test, y_test)

CPU times: user 1.17 s, sys: 713 ms, total: 1.88 s
Wall time: 294 ms


0.8763174603174603

In [7]:
# %%time
# from sklearn.svm import SVC

# svc = SVC()
# svc.fit(x_train, y_train)
# svc.score(x_test, y_test)

In [8]:
%%time
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test, y_test)

CPU times: user 333 ms, sys: 297 ms, total: 630 ms
Wall time: 187 ms


0.8763174603174603

In [18]:
estimators=[('knn', knn), ('rf', rf), ('log_reg', log_reg), ]
# #create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

# #fit model to training data
ensemble.fit(x_train, y_train)
# #test our model on the test data
ensemble.score(x_test, y_test)

0.8977619047619048

In [19]:
preds = ensemble.predict(X_test)

In [20]:
sub_lgb = pd.DataFrame()
sub_lgb['id'] = test['id']
sub_lgb['risk_flag'] = preds
import datetime
time = datetime.datetime.now() 
name = f"ensemble_{time}.csv"
sub_lgb.head()

,id,risk_flag
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [21]:
sub_lgb.risk_flag.value_counts()

0    25609
1     2391
Name: risk_flag, dtype: int64

In [22]:
sub_lgb.to_csv(name, index=False)
